In [6]:
from tqdm import tqdm
import itertools
import numpy as np
from snpp.utils.matrix import load_sparse_csr, \
    split_train_test
from snpp.utils.signed_graph import matrix2graph


dataset = 'slashdot'
raw_mat_path = 'data/{}.npz'.format(dataset)
random_seed = 123456

In [2]:
m = load_sparse_csr(raw_mat_path)

print('split_train_test')
train_m, test_m = split_train_test(
    m,
    weights=[0.9, 0.1])

test_entries = set(tuple(sorted((i, j)))
                   for i, j in zip(*test_m.nonzero()))
g = matrix2graph(m, None)

split_train_test


708it [00:00, 7070.57it/s]

building MultiGraph


516575it [01:26, 5950.18it/s]


In [3]:
# getting triangles
nodes_nbrs = g.adj.items()

triangles = set()
for v, v_nbrs in tqdm(nodes_nbrs):
    vs = set(v_nbrs) - set([v])
    ntriangles = 0
    for w in vs:
        ws = set(g[w]) - set([w])
        for u in vs.intersection(ws):
            triangles.add(tuple(sorted([u, v, w])))
        
print("{} triangles".format(len(triangles)))

100%|██████████| 77350/77350 [00:26<00:00, 2892.37it/s] 

548054 triangles


In [9]:
cnt = np.zeros(4)
for t in triangles:
    its = filter(lambda e: tuple(sorted(e)) in test_entries,
                 itertools.combinations(t, 2))
    cnt[len(list(its))] += 1

print('the percetange of 0, 1, 2, 3-order triangles:')
print(cnt / np.sum(cnt) * 100)

the percetange of 0, 1, 2, 3-order triangles:
[ 65.93748061  29.30112726   4.51451864   0.24687348]
